In [ ]:
# ENTER YOUR NAME AND ID HERE
# Sangeon Park
# A11733433

# Project-2: Locality Sensitive Hashing

In [1]:
import random
import os
import numpy as np
import re
import string
import regex
import sys
from functools import reduce

from random import randrange
from shutil import copyfile

Suspicious datasets for this project will be in <b>Suspicious</b> directory


Your query datasets will be in <b>Original</b> directory


You have to use any one original Wikipedia article from <b>Original</b> for <b>Fact Checks</b> steps

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Data/corpus-20090418'

os.makedirs('Suspicious_Sample')
os.makedirs('Original_Sample')
# you don't have to run this it is just organize original and suspicious files into seperate directory.
for file in os.listdir(path):
  if 'orig' in file:
    copyfile(path + '/'+file,'Original_Sample/'+file)
  else: 
    copyfile(path + '/'+file,'Suspicious_Sample/'+file)

### STEP - 1: Shingling (20 points)

In [4]:
# Type your code here... 
# Create necessary number of cells below this cell
susdir = '/content/Suspicious_Sample'
oridir = '/content/Original_Sample'
preprocessing_suslist = []
filename_suslist = []

susfilelist = os.listdir(susdir)
# print(susfilelist)
for file in sorted(susfilelist):
  with open(susdir+"/"+file,'r',encoding = "ISO-8859-1") as file1:
    file1 = file1.read()
    preprocessing = file1.replace('\n','')
    # Switch upper->lower
    preprocessing = file1.lower()
    preprocessing = re.sub('[^a-z0-9 ]+ ','',preprocessing)

    # # remove special character
    pattern = r'[' + string.punctuation + ']'
    preprocessing = re.sub(pattern,'',preprocessing)
    # # remove latin word
    preprocessing = re.sub(r'[^\x00-\x7f]',r'',preprocessing)
    # # print(file ,preprocessing)
    # one document preprocessing done
    preprocessing_suslist.append(preprocessing)
  # appending file name into list
  filename_suslist.append(file)
# print(filename_suslist)
# print(preprocessing_suslist)
# NOTE: No complex text processing is required
# convert just upper case characters to lower case


In [5]:
def gettotalKshingles(preprocessing_suslist,k):
  Kshingles_total= set()
  # for loop for reading single suspicious 
  for single_sus in preprocessing_suslist:
    # split by word split default is space. 
    split_word = single_sus.split()
    shingles = []
    # run list by len(split_word)-k-1. -k-1 is for prevent out of bounds index
    for i in range(0,len(split_word)-k-1):
      # grouping the words based on value of k
      shingles.append(tuple(split_word[i:i+k]))
    # changing to set remove duplicate k shingle  
    shingles = set(shingles)
    # update(add) into total list
    Kshingles_total.update(shingles)
    #return total k-shingles for all document
  return Kshingles_total

# 
def getKshingles_perfile(preprocessing_suslist,k):
  Kshingles_perfile= list()
  # for loop for reading single suspicious 
  for single_sus in preprocessing_suslist:
    # split by word split default is space. 
    split_word = single_sus.split()
    shingles = []
    # run list by len(split_word)-k-1. -k-1 is for prevent out of bounds index
    for i in range(0,len(split_word)-k-1):
      # grouping the words based on value of k
      shingles.append(tuple(split_word[i:i+k]))
    # changing to set remove duplicate k shingle  
    shingles = set(shingles)
    # append(add) into total list. it can save seperately by document. Above function is for gathering all shingles in one document. 
    Kshingles_perfile.append(shingles)
  return Kshingles_perfile

# print("k=3", len(gettotalKshingles(preprocessing_suslist,3)))
# print("k=4", len(gettotalKshingles(preprocessing_suslist,4)))
# print("k=5", len(gettotalKshingles(preprocessing_suslist,5)))
# print(gettotalKshingles(preprocessing_suslist,5))

In [6]:
# Type your code to get the 5-shingle index here
# to build signature matrix
shingles_total = gettotalKshingles(preprocessing_suslist,5)
Shingles5_with_index = dict()
# provide shingles with index
for index, value in enumerate(shingles_total):
  Shingles5_with_index[index] = value
# print(Shingles5_with_index)

### STEP - 2: Min-Hashing (40 points)

In [7]:
# ***************************************************NEW***********************************************************
# Generate Hash functions - 
    # We use (ax + b) mod N formula to permute shingle index
    # where a,b are random numbers, N total index size, and x is the index
# We need to do L permutations - In other words we need to have L permutations (lists) of new indexes
# Following function takes total index size N and L as arguments
    # And returns L new lists of size N    
def get_hash_functions(N,L):
    hash_functions = []
    
    for itr in range(L):
        a=randrange(1,400)
        b=randrange(1,400)
        
        new_hash_function = []
        for i in range(N):
            new_hash_function.append((a * i + b) % N)
        
        hash_functions.append(new_hash_function)
    return hash_functions
        
# test
# hash_functions = get_hash_functions(5000,50)
# print(hash_functions)

In [8]:
# Type your code here to generate all L hash functions
# Generate hash functions only for shingle index created for k=5
L = {50,100,200,500,1000}
N = len(shingles_total)
hash_functions_list = []

# permutation
# I used the for loop for appending and run get_hash_function into list but some reason it randomly mixed up. 
hash_functions_list.append(get_hash_functions(N,50))
hash_functions_list.append(get_hash_functions(N,100))
hash_functions_list.append(get_hash_functions(N,200))
hash_functions_list.append(get_hash_functions(N,500))
hash_functions_list.append(get_hash_functions(N,1000))


test_hash = get_hash_functions(N,50)
# print(hash_functions50)

In [ ]:
# print(test_hash)

In [10]:
# Type your code here to get the final signature matrix S here
# input matrix
################################# whatever commented code is testcase for one loop before I build def version #########################################
# f = open('stdout2.txt', 'w')
# shingles_per_file = getKshingles_perfile(preprocessing_suslist,5)
# shingles_total = gettotalKshingles(preprocessing_suslist,5)
# inputmatrix_perfile = []
# signatrue_matrix=np.full((len(test_hash),len(shingles_per_file)),9999999999)


# for col,fileShingles in enumerate(shingles_per_file):
#   for i in shingles_total:
#     temp = {i}
#     if temp.issubset(fileShingles):
#       inputmatrix_perfile.append(1)
#     else:
#       inputmatrix_perfile.append(0)
#   for index ,onebitvector in enumerate(inputmatrix_perfile):
#     if onebitvector ==1:
#       for row, hash in enumerate(test_hash):
#         if hash[index] < signatrue_matrix[row][col]:
#           signatrue_matrix[row][col] = hash[index]
#   inputmatrix_perfile = []

# np.set_printoptions(threshold=np.inf)
# print(inputmatrix_perfile)
# print(signatrue_matrix,file=f)
# f.close()

def makeSignatureMatrix(shingles_per_file,shingles_total,All_hash):
  # shingles_per_file = getKshingles_perfile(preprocessing_suslist,5)
  # shingles_total = gettotalKshingles(preprocessing_suslist,5)
  inputmatrix_perfile = []
  signatrue_matrix=np.full((len(All_hash),len(shingles_per_file)),9999999999)

  for col,fileShingles in enumerate(shingles_per_file):#loop by documents(95)
    for i in shingles_total:
      # i is tuple type. put i in set
      temp = {i}
      # if temp(one shingles) in one file shingles list append 1 otherwise append 0 into list.
      if temp.issubset(fileShingles):
        inputmatrix_perfile.append(1)
      else:
        inputmatrix_perfile.append(0)
    # after complete one document inputmatrix(1,amount of shingles for one document). run that inputmatrix
    for index ,onebitvector in enumerate(inputmatrix_perfile):
      if onebitvector ==1:
        # loop the permutation for inputmatrix status is 1
        for row, hash in enumerate(All_hash):
          # compare with the signature matrix value(inital should large int), if it find the smaller value from the permutation than signature value 
          if hash[index] < signatrue_matrix[row][col]:
            # replace it
            signatrue_matrix[row][col] = hash[index]
    # After finish to make signature matrix for one document, reset the one document input matrix
    inputmatrix_perfile = []
  return signatrue_matrix

# f = open('stdout3.txt', 'w')
# np.savetxt('test_hash.txt',signatrue_matrix,delimiter=',')
# np.set_printoptions(threshold=np.inf)
# result = makeSignatureMatrix(test_hash)
# print(result,file=f)
signature_matrix_list=[]
shingles_per_file = getKshingles_perfile(preprocessing_suslist,5)
shingles_total = gettotalKshingles(preprocessing_suslist,5)
for i in hash_functions_list:
  signature_matrix_list.append(makeSignatureMatrix(shingles_per_file,shingles_total,i))

In [ ]:
# a = np.full((2,3),1)
# print(a)

(50, 95)
50 = row
95 = colum

(2,3)
[[1 1 1]
 [1 1 1]]

In [11]:
# Type your code here to do the fact check 
#      with any one query document in the 'Original' directory
############## this process is exacly same that when I preprocessing and append into list in suspicious list 
susdir = '/content/Suspicious_Sample'
oridir = '/content/Original_Sample'
preprocessing_orilist = []
filename_orilist = []

orifilelist = os.listdir(oridir)
# print(susfilelist)
for file in sorted(orifilelist):
  with open(oridir+"/"+file,'r',encoding = "ISO-8859-1") as file1:
    file1 = file1.read()
    file1 = file1.replace('\n','')
    # Switch upper->lower
    preprocessing = file1.lower()
    preprocessing = re.sub('[^a-z0-9 ]+ ','',preprocessing)
    # # remove special character
    pattern = r'[' + string.punctuation + ']'
    preprocessing = re.sub(pattern,'',preprocessing)
    # # remove latin word
    preprocessing = re.sub(r'[^\x00-\x7f]',r'',preprocessing)
    preprocessing_orilist.append(preprocessing)
  filename_orilist.append(file)
# print(filename_orilist)
# print(preprocessing_orilist)

# STEP-1: Generate 5-shingles 
    # (if any shingles are not present in your shingle index, simply ignore them)
# this process is same as when program make suspcious shingles
total_orginal_five_shingles = gettotalKshingles(preprocessing_orilist,5)
per_file_original_five_shingles = getKshingles_perfile(preprocessing_orilist,5)
# print(per_file_original_five_shingles)    




# STEP-2: Generate signature vector from L hash functions
original_signature_list = []
# this process is exactly same as when program create suspicous signature matrix 
for i in hash_functions_list:
  original_signature_list.append(makeSignatureMatrix(per_file_original_five_shingles,shingles_total,i))

In [17]:
# STEP-3: Calculate Jaccard similarity of signature vector of orginal doc.

# default setting is .6 but as you see the result shows up to .90+
t = 0.6
total_over_85=[]

# five_ori_signaturematrix = 5개의 시그니쳐 매트릭스중 하나
# 5 different kind of L(50,100,200,500,1000) run 5 times
for index,five_ori_signaturematrix in enumerate(original_signature_list):
        # if L=50, (50,95), one of the fifth signature matrix
        ninefive_sus_signaturematrix = signature_matrix_list[index]
        # access by row(amount of total shingles)
        for ori_counter in range(five_ori_signaturematrix.shape[1]):
                #(50,) 
                # one document signaturematrix
                one_ori_signaturematrix = five_ori_signaturematrix[:,ori_counter]
                for sus_counter in range(ninefive_sus_signaturematrix.shape[1]):
                          one_sus_signaturematrix = ninefive_sus_signaturematrix[:,sus_counter]
                          # countX is increament when one original signature vector and one suspicious signature vector is match
                          countX = 0
                          # countY is increament when one original signature vector and one suspicious signature vector is not match
                          countY = 0
                          # print(one_ori_signaturematrix)
                          # print(one_sus_signaturematrix)
                          # break
                          for compare_coutner, one_sus_value in enumerate(one_sus_signaturematrix):
                            #  print(one_ori_signaturematrix.shape)
                            #  print(one_sus_signaturematrix.shape)
                            #  break
                            # check by row so do samething on original 
                             one_ori_value = one_ori_signaturematrix[compare_coutner]
                            #  print(one_sus_value)
                            #  print(one_ori_value)
                            #  break
                             if one_sus_value == one_ori_value:
                               countX = countX+1
                             else:
                               countY = countY + 1
                          # similiarity equation(intersecion of two matrix/ union of two matrix)
                          jaccard_similiarity = countX/(countX+countY)
                          # if the similiarity is greater than t value
                          if jaccard_similiarity > t:
                                # save into total list
                                temp=[filename_orilist[ori_counter],filename_suslist[sus_counter],jaccard_similiarity,five_ori_signaturematrix.shape[0]]
                                total_over_85.append(temp)
            

# STEP-4: Facts Checks
# For each L = {50,100,200,500,1000}, report all documents (file_names) below that have Jaccard similarity > t=0.85
# Sort the documents in decreasing order of the Jaccard similarity


# sort by similiarity with descending order
sorted_over85=sorted(total_over_85,reverse=True, key = lambda a: a[2])
for i in sorted_over85:
    print("original document : ",i[0]," suspicious document : ",i[1]," similiarity : ",i[2]," L = ",i[3])     


original document :  orig_taska.txt  suspicious document :  g0pE_taska.txt  similiarity :  0.94  L =  50
original document :  orig_taska.txt  suspicious document :  g0pE_taska.txt  similiarity :  0.92  L =  100
original document :  orig_taska.txt  suspicious document :  g0pE_taska.txt  similiarity :  0.902  L =  500
original document :  orig_taska.txt  suspicious document :  g0pE_taska.txt  similiarity :  0.892  L =  1000
original document :  orig_taska.txt  suspicious document :  g0pE_taska.txt  similiarity :  0.885  L =  200
original document :  orig_taskd.txt  suspicious document :  g4pC_taskd.txt  similiarity :  0.84  L =  100
original document :  orig_taskd.txt  suspicious document :  g3pA_taskd.txt  similiarity :  0.82  L =  50
original document :  orig_taska.txt  suspicious document :  g4pC_taska.txt  similiarity :  0.82  L =  100
original document :  orig_taska.txt  suspicious document :  g4pC_taska.txt  similiarity :  0.808  L =  500
original document :  orig_taska.txt  suspic

### STEP - 3: LSH (30 points)

In [13]:
# Type your code here to hash signature matrix into B buckets
# Use the technique to split the signature matrix into b bands of r rows
# Convert only the signature matrix generated with L=1000

# base setting for LSH all explaination is given above
b = 50
r = 20
B = 199

N = len(shingles_total)
permutation =get_hash_functions(N,1000)

sus_shingles_total = gettotalKshingles(preprocessing_suslist,5)
sus_shingles_per_file = getKshingles_perfile(preprocessing_suslist,5)
sus_signature_matrix = makeSignatureMatrix(sus_shingles_per_file,sus_shingles_total,permutation)


In [14]:
# same preprocssing logic like previous cases but this is for one file preprocessing

oridir = '/content/Original_Sample'
def onefile_preprocessing(file,oridir):
    with open(oridir+"/"+file,'r',encoding = "ISO-8859-1") as file1:
      file1 = file1.read()
      file1 = file1.replace('\n','')
      # Switch upper->lower
      preprocessing = file1.lower()
      preprocessing = re.sub('[^a-z0-9 ]+ ','',preprocessing)
      # # remove special character
      pattern = r'[' + string.punctuation + ']'
      preprocessing = re.sub(pattern,'',preprocessing)
      # # remove latin word
      preprocessing = re.sub(r'[^\x00-\x7f]',r'',preprocessing)
    return [preprocessing]

In [31]:
from random import sample
# Type your code here to do generate candidate documents

# this def make input and signature matrix


# slice the signature matrix by given band and row
# since L=1000 which means row is 1000. one band = 20 rows and that band will have 50 bands.
def make_band_row_matrix(sus_signature_matrix,r):
  row_counter=1
  totalband = []
  oneband=[]
  # run sus_signature_matrix by one row
  for one_row in sus_signature_matrix:
    # since oneband = 20 band, make sublist(oneband) and append row 20 times
    if row_counter < r :
      oneband.append(one_row)
      row_counter = row_counter + 1
    # if one band full with 20 rows, reset the sublist(one band)  and append into totalband list.
    else:
      oneband.append(one_row)
      # reset row_counter to use for next band
      row_counter = 1
      totalband.append(oneband)
      # reset oneband to use for next band
      oneband = []
    
  return totalband
# logic is exactly same as below def but this is for suspcious bucket.
def assign_sus_to_bucket(band_row_matrix,r,B,random_one_value_list):
  sus_into_bucket_dict = dict()
  # since band_row_matrix has 50 bands want to check by one band
  for oneband in band_row_matrix:
    # convert list to numpy.array
    oneband = np.array(oneband)
    # oneband = (20,95)
    total_col = len(oneband[0])#95
    # run 95 times since we have 95 suspcious document
    for col in range(total_col):
      # initialize rixai
      total_rixai = 0
      one_doc_rows=oneband[:,col]#(20,1)
      # loop one value in the 20 rows
      for index,one_value in enumerate(one_doc_rows):
        # this process is pretty straight forward because this is given in the instruction. 
        ri = one_value
        ai = random_one_value_list[index]
        rixai = ri * ai
        total_rixai = total_rixai + rixai
      h = total_rixai%B# this will be address for bucket. bucket 1~199
      # if bucket address(h) is not exist in the bucket,
      if h not in sus_into_bucket_dict:
        # create key = h, value = empty
        sus_into_bucket_dict[h] = []
      # if bucket address is existed in the ori_into_bucket_dict,
      if col in sus_into_bucket_dict:
      # do nothing
        continue
      # put filename for better visualization into key = h
      else:
        sus_into_bucket_dict[h].append(col)
      # print(h)
      # break
    # print(sus_into_bucket_dict)
    # break
  return sus_into_bucket_dict

# logic is exactly same as above def but this is for orignal bucket.
def assign_ori_to_bucket(band_row_matrix,filename,r,B,random_one_value_list):
  ori_into_bucket_dict = dict()
  # since band_row_matrix has 50 bands want to check by one band
  for oneband in band_row_matrix:
    # convert list to numpy.array
    oneband = np.array(oneband)
    # oneband = (20,95)
    total_col = len(oneband[0])#95
    # run 95 times since we have 95 suspcious document
    for col in range(total_col):
      total_rixai = 0
      one_doc_rows=oneband[:,col]#(20,1)
      # loop one value in the 20 rows
      for index,one_value in enumerate(one_doc_rows):
        # this process is pretty straight forward because this is given in the instruction. 
        ri = one_value
        ai = random_one_value_list[index]
        rixai = ri * ai
        total_rixai = total_rixai + rixai
      h = total_rixai%B# this will be address for bucket. bucket 1~199
      # if bucket address(h) is not exist in the bucket,
      if h not in ori_into_bucket_dict:
      # create key = h, value = empty
        ori_into_bucket_dict[h] = []
      # if bucket address is existed in the ori_into_bucket_dict,
      if filename in ori_into_bucket_dict:
      # do nothing
        continue
      # put filename for better visualization into key = h
      else:
        ori_into_bucket_dict[h].append(filename)
      # print(h)
      # break
    # print(ori_into_bucket_dict)
    # break
  return ori_into_bucket_dict

def jaccard_similiarity(sus_signature_matrix, ori_signature_matrix ,candidates_document_list ,t):
  # initialize false positive and false negative count
  count_False_Positive = 0
  count_False_Negative = 0
  candidates_document_list_range = len(candidates_document_list)
  # there should be multiple candidate document in the list so read by one document
  for one_document_counter in range(candidates_document_list_range):
    # strucutre of logic is similiar as previous jarcard smiliarity in min hashing
    one_document = candidates_document_list[one_document_counter]
    # countX is increament when one original signature vector and one suspicious signature vector is match
    countX = 0
    # countY is increament when one original signature vector and one suspicious signature vector is not match
    countY = 0
    for index, one_sus_signature_vector in enumerate(sus_signature_matrix[:,one_document]): 
      # if one ori_signature_vector == one_sus_signature_vector
      if ori_signature_matrix[:,0][index] == one_sus_signature_vector:
        countX = countX + 1
      else:
        countY = countY + 1
    # similiarity equation(intersecion of two matrix/ union of two matrix)
    jaccard_similarity = countX/(countX+countY)
    # if similiartity is less than equal to t value,
    if jaccard_similarity <= t:
      # increament count_False_Positive
      count_False_Positive = count_False_Positive + 1
    # if similiartity is greater than t value,
    else:
      # increament count_False_Negative
      count_False_Negative = count_False_Negative + 1
  return count_False_Positive, count_False_Negative


# this list provide amount number of first argument without duplication
random_one_value_list = sample(range(r),r)
# make suspicious matrix using suspicious signature matrix
sus_band_row_matrix = make_band_row_matrix(sus_signature_matrix,r)
# using sus_band_row_matrix to make suspcious bucket
sus_to_bucket = assign_sus_to_bucket(sus_band_row_matrix,r,B,random_one_value_list)

#read one orignal filename in the filename_orilist
for one_ori_filename in filename_orilist:
  # preprocessing one orignal document
  onefile_preprocessed = onefile_preprocessing(one_ori_filename,oridir)
  # make shingles total for one original document
  ori_shingles_total = gettotalKshingles(onefile_preprocessed,5)
  # make shingles perfile(there will be only one but need for to reuse def) for one original document
  ori_shingles_per_file = getKshingles_perfile(onefile_preprocessed,5)
  # make one original signature matrix
  ori_signature_matrix = makeSignatureMatrix(ori_shingles_per_file,sus_shingles_total,permutation)
  # make original matrix using original signature matrix
  ori_band_row_matrix = make_band_row_matrix(ori_signature_matrix,r)
  # using ori_band_row_matrix to make original bucket
  ori_to_bucket = assign_ori_to_bucket(ori_band_row_matrix,one_ori_filename,r,B,random_one_value_list)

  #initialize candidate list
  candidate_list = []

  # compare original bucket and suspcious bucket
  for key in ori_to_bucket.keys():
    #if original key is in the suspcious bucket dictionary
    if key in sus_to_bucket.keys():
      # add into candidate list
      candidate_list.extend(sus_to_bucket[key])
  # convert to list to set that make sure there is no duplication
  candidates_document = set(candidate_list)

  candidates_document_list = list(candidates_document)
  # print(ListOfcandidatesDoc)
  t=0.6
  print(one_ori_filename,"\n")
  false_positive, false_negative = jaccard_similiarity(sus_signature_matrix,ori_signature_matrix,candidates_document_list,t)
  print("False Positives: ",false_positive)
  print("False Negatives: ", false_negative)
  print("\n")



# Follow all steps from STEP - 2 fact check (except the Jaccard similarity part)

# STEP - 1: Split each original document signature vector into b bands of r rows

# STEP - 2: Hash using the same hash functions created for 
    # signature matrix hashing (in the previous cell)
    

orig_taska.txt 

False Positives:  26
False Negatives:  1


orig_taskb.txt 

False Positives:  31
False Negatives:  0


orig_taskc.txt 

False Positives:  31
False Negatives:  0


orig_taskd.txt 

False Positives:  34
False Negatives:  1


orig_taske.txt 

False Positives:  34
False Negatives:  0




In [ ]:
# Type your code here to do the fact check
# Calculate Jaccard similarity of the oiginal document with only 
    # candidate documents
    